In [8]:
import json
import random
from IPython.display import clear_output
from threading import Thread, Lock
from datetime import datetime
import time

mutex=Lock()

In [9]:
with open('./json_info_sezioni.json') as json_file:
    info_sezioni = json.load(json_file)

In [10]:
with open('./json_posti_lavoro.json') as json_file:
    posti_lavoro = json.load(json_file)

In [11]:
with open('./POI_scuola.json') as json_file:
    scuole = json.load(json_file)

In [12]:
#Dati pendolarismo Milano
with open('/Volumes/HDD/Università/Tirocinio/Dati censo/ds390_Pendolarismo_entroMI_sezione.json') as json_file:
    spostamenti = json.load(json_file)

In [14]:
print("Inizio =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

agenti=[]
id_agente=0
count_spost=0
mov_non_elab=[]

def crea_agente(s):
    
    global agenti
    global id_agente
    global mov_non_elab
    
    for i in range(0,s['Numerositŕ']):

        agente={} 
        sez_origine=s['Sezione di origine']
        sez_destinazione=s['Sezione di destinazione']

        if sez_origine =='Non determinata' or sez_destinazione=="Non determinata":
            break
        
        sez=list(filter(lambda sez: sez['sezione'] == int(sez_origine), info_sezioni))
        if sez:
            sez=sez[0]
        else:
            break
        
        posti_disp=list(filter(lambda p: p['sezione_censo'] == int(sez_destinazione), posti_lavoro))
        random.shuffle(posti_disp)
        scuole_disp=list(filter(lambda scuola: scuola['sezione_censo'] == int(sez_destinazione), scuole))
        random.shuffle(scuole_disp)

        if s['Motivo']=='Lavoro':
            for p in posti_disp:
                if sez['num_eta_attiva_15-64']>0 and p['dipendenti']>0:
                    sez['num_eta_attiva_15-64']=sez['num_eta_attiva_15-64']-1
                    p['dipendenti']=p['dipendenti']-1

                    agente={}
                    agente['id']=id_agente
                    id_agente=id_agente+1
                    agente['tipo']='attiva'
                    agente['categoria']='lavoratore'
                    agente['sez_residenza']=sez['sezione']
                    agente['posto_lavoro']=p['id']
                    agente['automunito']=random.choice([True, False])
                    
                    agenti.append(agente)
                    break
            if agente =={}:
                mov_non_elab.append('Spost: '+str(count_spost)+' Orig: '+str(sez_origine)+' Dest: '+str(sez_destinazione)+' Motivo: '+s['Motivo'])  

        if s['Motivo']=='Studio':
            for p in scuole_disp:                    
                if sez['num_giovani_5-14']>0 and p['capienza']>0 and (p['tipo_edificio']=='school' or p['tipo_edificio']=='kindergarten'):
                    sez['num_giovani_5-14']=sez['num_giovani_5-14']-1
                    p['capienza']=p['capienza']-1

                    agente={}
                    agente['id']=id_agente
                    id_agente=id_agente+1
                    agente['tipo']='giovane'
                    agente['categoria']='studente'
                    agente['sez_residenza']=sez['sezione']
                    agente['scuola_appartenenza']=p['id']
                    agente['automunito']=False

                    agenti.append(agente)
                    break

                if sez['num_eta_attiva_15-64']>0 and p['capienza']>0 and (p['tipo_edificio']=='university' or p['tipo_edificio']=='college'):
                    sez['num_eta_attiva_15-64']=sez['num_eta_attiva_15-64']-1
                    p['capienza']=p['capienza']-1

                    agente={}
                    agente['id']=id_agente
                    id_agente=id_agente+1
                    agente['tipo']='attiva'
                    agente['categoria']='studente'
                    agente['sez_residenza']=sez['sezione']
                    agente['scuola_appartenenza']=p['id']
                    agente['automunito']=random.choice([True, False])

                    agenti.append(agente)
                    break

            if agente =={}:
                mov_non_elab.append('Spost: '+str(count_spost)+' Orig: '+str(sez_origine)+' Dest: '+str(sez_destinazione)+' Motivo: '+s['Motivo'])  
            
for s in spostamenti:
    crea_agente(s)

print('Spostamenti validi: '+str(len(agenti)))
print('Spostamenti non elaborati: '+str(len(mov_non_elab)))

print("Fine =", datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

Inizio = 2021-03-23 11:32:43
Spostamenti validi: 210737
Spostamenti non elaborati: 292621
Fine = 2021-03-23 12:22:02


In [15]:
with open('./json_agenti.json', 'w') as fp:
        json.dump(agenti, fp)

In [16]:
with open('./json_non_elab.json', 'w') as fp:
        json.dump(mov_non_elab, fp)

In [17]:
with open('./json_posti_lavoro_rimasti.json', 'w') as fp:
        json.dump(posti_lavoro, fp)

In [18]:
with open('./json_scuole_rimaste.json', 'w') as fp:
        json.dump(scuole, fp)